Principal component analysis is a linear transformation technique that has applications in several domains, most commonly in dimensionality reduction and data compression. PCA identifies the correlation between the features and tries to remove the redundant or less useful features. In essence, it finds the subspace that can best represent the high dimensional data. Some other commonly used dimensionality reduction techniques are - Linear Discriminant Analysis (LDA), t-SNE and Autoencoders.

I have implemented both PCA and LDA from scratch. Using the PCA technique, I also demonstrate the reconstruction of the original images from the sum of first k principal components. Similar experimenting with LDA has also been done.

In [ ]:
import numpy as np
import os
import cv2
from scipy import ndimage
import scipy
import time
from math import sqrt, cos, pi
import matplotlib 
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import StandardScaler
import pickle

In [ ]:
class PCA_LDA_SNE_AEN_Analysis:

    def __init__(self):
        self.train_data = []
        self.train_labels = []
        self.test_data = []
        self.test_labels = []
        self.data_classes = {}
    
    def unpickle(self, file):
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict

    def load_data(self):
        # Load the CIFAR dataset, there are 5 batches of training images, let's load
        # each of them and append them together
        print('loading data of batch: ')
        for i in range(5):
            print(str(i+1))
            this_data = self.unpickle('dataset_CIFAR/data_batch_'+str(i+1))
            # the dictionary has the following elements - batch_label, labels, data and filenames
            # we only need the data and the labels
            if i == 0:
                self.train_data = this_data[b'data']
                self.train_labels = this_data[b'labels']
                continue
            self.train_data = np.vstack((self.train_data, this_data[b'data']))
            self.train_labels.extend(this_data[b'labels'])
        # now load the test data
        test_all_data = self.unpickle('dataset_CIFAR/test_batch')
        self.test_data = test_all_data[b'data']
        self.test_labels = test_all_data[b'labels']
        # this data_class dict object in turn has following elements - num_cases_per_batch,
        # label_names and num_vis. We may need only label_names mostly, hence taking only that
        self.data_classes = self.unpickle('dataset_CIFAR/batches.meta')[b'label_names']
        return

    def histo_visual(self):
        with plt.style.context('seaborn-whitegrid'):
            plt.figure(figsize=(8, 6))
            #visualize the first ten features for all categories
            colors = cm.rainbow(np.linspace(0, 1, 10))
            for cnt in range(10):
                print('feature '+ str(cnt+1))
                plt.subplot(5, 2, cnt+1)
                for lab in range(len(self.data_classes)):
                    # indi = np.where(np.array(self.train_labels)==lab)
                    indi = [i for i in range(len(self.train_labels)) if self.train_labels[i] == lab]
                    plt.hist(self.train_data[indi, cnt],
                             bins=10,
                             )
                plt.xlabel('feature ' + str(cnt+1))
            plt.legend(loc='upper right', fancybox=True, fontsize=8)
            plt.tight_layout()
            plt.show()
        return

In [ ]:
def pca_f(self):
        self.load_data()
        '''#check to see if 50000 training images and 10000 test images with labels have been loaded
        print(self.train_data.shape)
        print(len(self.train_labels))
        print(self.test_data.shape)
        print(len(self.test_labels))
        print(self.data_classes)'''
        # plot a histogram of the first 10 features
        self.histo_visual()
        '''# standardize the image:
        self.train_data = StandardScaler().fit_transform(self.train_data)
        mean_vec = np.mean(self.train_data, axis=0)
        covar = (self.train_data - mean_vec).T.dot((self.train_data - mean_vec)) / (self.train_data.shape[0]-1)'''
        covar = np.cov(self.train_data.T)
        print('Covariance matrix \n ', covar)
        print('shape of covar matrix ', covar.shape)

        #option 1 - eigen value decomposition
        eigv, eig_vec = np.linalg.eig(covar)

        #option 2 - singular value decomposition
        '''u,s,v = scipy.linalg.svd(self.train_data.T.astype(np.float32))
        print(v)'''

        #(eigen value, eigen vector) tuples
        eigen_all = [(np.abs(eigv[i]), eig_vec[:,i]) for i in range(len(eigv))]
        # Sort the (eigenvalue, eigenvector) tuples from high to low
        eigen_all.sort(key=lambda y: y[0], reverse=True)

        # check the sorted order
        '''print('Sorted top 10 Eigenvalues:')
        k = 0
        for j in eigen_all:
            print(j[0])
            k+=1
            if k>= 10 :
                break'''
        # determine the value of K for selecting the first k principal components
        eigv_sum = sum(eigv)
        variance_ = [(i / eigv_sum)*100 for i in sorted(eigv, reverse=True)]
        print(variance_[:10])
        cum_var = np.cumsum(variance_)

        with plt.style.context('seaborn-whitegrid'):
            plt.figure(figsize=(6, 4))
            '''plt.bar(range(len(eigv)), variance_, alpha=0.5, align='center',
                    label='individual explained variance')
            plt.step(range(len(eigv)), cum_var, where='mid',
                     label='cumulative explained variance')'''
            plt.bar(range(10), variance_[:10], alpha=0.5, align='center',
                    label='individual explained variance')
            plt.step(range(10), cum_var[:10], where='mid',
                     label='cumulative explained variance')
            plt.ylabel('Explained variance ratio')
            plt.xlabel('Principal components')
            plt.legend(loc='best')
            plt.tight_layout()
            plt.show()
        # plot the first 2 principal components and visualize the data separation
        # (use inbuilt pca from scikit to fit and then visualize)
        
        # show the original images (one image from each of the 10 categories)
        num_eg = 5
        indi = [np.where(np.array(self.train_labels)==lab)[:num_eg] for lab in range(0, len(self.data_classes))]
        # plotting the images
        fig, aa = plt.subplots(len(self.data_classes),num_eg,figsize=(32, 32))
        for class_id in range(len(self.data_classes)):
            for i in range(num_eg):
                #aa[class_id,i].imshow(np.reshape(self.train_data[indi[class_id][0][i]],(32,32,3)))
                aa[class_id, i].imshow(np.dstack((self.train_data[indi[class_id][0][i]][:1024].reshape((32,32,1)),self.train_data[indi[class_id][0][i]][1024:2048].reshape((32,32,1)),self.train_data[indi[class_id][0][i]][2048:].reshape((32,32,1)))))
                aa[class_id,i].axis('off')

        fig.suptitle('{} sample images from each class'.format(5))
        plt.show()
        # reconstruct the image using first 10 principal components (of the same 10 images):
        # find mean image from training data set:
        train_data_mean = np.mean(self.train_data, axis=0)
        mean_sub_data = self.train_data - train_data_mean
        # now reconstruct the data for each image with the 10 eigen vectors
        coeff = []
        top_ten_eig = []
        for i in range(200):
            this_coeff = np.matmul(mean_sub_data, eigen_all[i][1])
            if i==0:
                coeff = this_coeff
                top_ten_eig = eigen_all[i][1]
                continue
            coeff = np.vstack((coeff, this_coeff))
            top_ten_eig = np.vstack((top_ten_eig, eigen_all[i][1]))
        # shape of coeff was 200 x 50000, now changing it to 50,000 x 200
        coeff = coeff.T
        print(coeff.shape)
        #shape of top_ten_eig is 200 x 3072
        recon_img = np.matmul(coeff,top_ten_eig)
        recon_img += train_data_mean
        print(recon_img.shape)
        #plot reconstructed Images
        num_eg = 5
        indi = [np.where(np.array(self.train_labels)==lab)[:num_eg] for lab in range(0, len(self.data_classes))]
        # plotting the images
        fig, aa = plt.subplots(len(self.data_classes),num_eg,figsize=(32, 32))
        for class_id in range(len(self.data_classes)):
            for i in range(num_eg):
                aa[class_id, i].imshow(np.dstack((recon_img[indi[class_id][0][i]][:1024].reshape((32,32,1)),recon_img[indi[class_id][0][i]][1024:2048].reshape((32,32,1)),recon_img[indi[class_id][0][i]][2048:].reshape((32,32,1)))).astype(np.uint8))
                aa[class_id,i].axis('off')

        fig.suptitle('{} reconstructed images from each class'.format(5))
        plt.show()
        return

In [ ]:
def lda_f(self):
        self.load_data()
        # first, let's compute the mean vectors of each class
        mean_vectors = []
        for cl in range(10):
            indi = [np.where(np.array(self.train_labels)==cl)]
            mean_vectors.append(np.mean(np.reshape(self.train_data[indi], (self.train_data[indi].shape[1], self.train_data[indi].shape[2])), axis=0))
            print('Mean Vector class ', cl, ': ', mean_vectors[cl], '\n')
            
        # compute within class scatter matrix
        scatter_within = np.zeros((3072, 3072))
        for lab, meanv in zip(range(10), mean_vectors):
            print('label ', lab)
            scat_matrix = np.zeros((3072, 3072))
            indi = [np.where(np.array(self.train_labels)==lab)]
            meanv = meanv.reshape(3072, 1)
            for data in np.reshape(self.train_data[indi], (self.train_data[indi].shape[1], self.train_data[indi].shape[2])):
                data = data.reshape(3072,1)
                scat_matrix = np.dot(data-meanv, (data-meanv).T)
                #scat_matrix += (data-meanv).dot((data-meanv).T)
            scatter_within += scat_matrix
            
        print('within-class Scatter Matrix:\n', scatter_within)

        # compute between class scatter matrix
        overall_mean = np.mean(self.train_data, axis=0)
        scatter_between = np.zeros((3072, 3072))
        for lab,meanv in enumerate(mean_vectors):
            indi = [np.where(np.array(self.train_labels)==lab)]
            n = np.reshape(self.train_data[indi], (self.train_data[indi].shape[1], self.train_data[indi].shape[2])).shape[0]
            meanv = meanv.reshape(3072,1)
            overall_mean = overall_mean.reshape(3072,1)
            scatter_between += n * np.dot((meanv - overall_mean),(meanv - overall_mean).T)
        print('between-class Scatter Matrix:\n', scatter_between)

        #EVD:
        eigv, eig_vec = np.linalg.eig(np.linalg.inv(np.dot(scatter_within, scatter_between)))

        ##Rest is the same as PCA 
        # print 10 eigen vectors
        for i in range(10):
            eigvec_sc = eig_vec[:,i].reshape(3072,1)   
            print('\nEigenvector {}: \n{}'.format(i+1, eigvec_sc.real))
            print('Eigenvalue {:}: {:.2e}'.format(i+1, eigv[i].real))

        #(eigen value, eigen vector) tuples
        eigen_all = [(np.abs(eigv[i]), eig_vec[:,i]) for i in range(len(eigv))]
        # Sort the (eigenvalue, eigenvector) tuples from high to low
        eigen_all.sort(key=lambda y: y[0], reverse=True)

        # check the sorted order
        '''print('Sorted top 10 Eigenvalues:')
        k = 0
        for j in eigen_all:
            print(j[0])
            k+=1
            if k>= 10 :
                break'''

        print('Variance explained:\n')
        eigv_sum = sum(eigv)
        for i,j in enumerate(eigen_all):
            print('eigenvalue {0:}: {1:.2%}'.format(i+1, (j[0]/eigv_sum).real))

        with plt.style.context('seaborn-whitegrid'):
            plt.figure(figsize=(6, 4))
            '''plt.bar(range(len(eigv)), variance_, alpha=0.5, align='center',
                    label='individual explained variance')
            plt.step(range(len(eigv)), cum_var, where='mid',
                     label='cumulative explained variance')'''
            plt.bar(range(10), variance_[:10], alpha=0.5, align='center',
                    label='individual explained variance')
            plt.step(range(10), cum_var[:10], where='mid',
                     label='cumulative explained variance')
            plt.ylabel('Explained variance ratio')
            plt.xlabel('Principal components')
            plt.legend(loc='best')
            plt.tight_layout()
            plt.show()
            
        # plot the first 2 principal components and visualize the data separation
        # (use inbuilt pca from scikit to fit and then visualize)
        
        # show the original images (one image from each of the 10 categories)
        num_eg = 5
        indi = [np.where(np.array(self.train_labels)==lab)[:num_eg] for lab in range(0, len(self.data_classes))]
        # plotting the images
        fig, aa = plt.subplots(len(self.data_classes),num_eg,figsize=(32, 32))
        for class_id in range(len(self.data_classes)):
            for i in range(num_eg):
                #aa[class_id,i].imshow(np.reshape(self.train_data[indi[class_id][0][i]],(32,32,3)))
                aa[class_id, i].imshow(np.dstack((self.train_data[indi[class_id][0][i]][:1024].reshape((32,32,1)),self.train_data[indi[class_id][0][i]][1024:2048].reshape((32,32,1)),self.train_data[indi[class_id][0][i]][2048:].reshape((32,32,1)))))
                aa[class_id,i].axis('off')

        fig.suptitle('{} sample images from each class'.format(5))
        plt.show()
        # reconstruct the image using first 10 principal components (of the same 10 images):
        # find mean image from training data set:
        train_data_mean = np.mean(self.train_data, axis=0)
        mean_sub_data = self.train_data - train_data_mean
        # now reconstruct the data for each image with the 10 eigen vectors
        coeff = []
        top_ten_eig = []
        for i in range(200):
            this_coeff = np.matmul(mean_sub_data, eigen_all[i][1])
            if i==0:
                coeff = this_coeff
                top_ten_eig = eigen_all[i][1]
                continue
            coeff = np.vstack((coeff, this_coeff))
            top_ten_eig = np.vstack((top_ten_eig, eigen_all[i][1]))
        # shape of coeff was 10 x 50000, now changing it to 50,000 x 10
        coeff = coeff.T
        print(coeff.shape)
        #shape of top_ten_eig is 10 x 3072
        recon_img = np.matmul(coeff,top_ten_eig)
        recon_img += train_data_mean
        print(recon_img.shape)
        #plot reconstructed Images
        num_eg = 5
        indi = [np.where(np.array(self.train_labels)==lab)[:num_eg] for lab in range(0, len(self.data_classes))]
        # plotting the images
        fig, aa = plt.subplots(len(self.data_classes),num_eg,figsize=(32, 32))
        for class_id in range(len(self.data_classes)):
            for i in range(num_eg):
                aa[class_id, i].imshow(np.dstack((recon_img[indi[class_id][0][i]][:1024].reshape((32,32,1)),recon_img[indi[class_id][0][i]][1024:2048].reshape((32,32,1)),recon_img[indi[class_id][0][i]][2048:].reshape((32,32,1)))).astype(np.uint8))
                aa[class_id,i].axis('off')

        fig.suptitle('{} reconstructed images from each class'.format(5))
        plt.show()
        
        return

In [ ]:
def main(self, choice):
        options = {'pca':self.pca_f,
                   'lda':self.lda_f,
            }
        options[choice]()
        return

In [ ]:
if __name__ == "__main__":

    # creating an instance of the PCA class
    pca_class = PCA_LDA_SNE_AEN_Analysis()

    # Call to the main function
    # Arguments to the main function is the analysis type (PCA, LDA, t-SNE or Auto encoders)
    tic = time.time()
    print('Choices are : \n 1. PCA (pca), \n 2. lda (lda_f), \n 3. t_sne visualization (t_sne) and \n 4. Auto-encoders(aen)')
    choice_v = input("Enter the choice: ")
    pca_class.main(choice_v)
    toc = time.time() - tic
    print("Running time: " + str(toc))


The dataset that was used for this exercise is the CIFAR-10 dataset. This dataset has in total 60000 images of size (32, 32, 3) (RGB image), out of which 10000 images are test images. The different categories of the dataset are :
Airplane, Automobile, Bird, Cat, Deer, Dog, Frog, Horse, Ship and truck.

There are around 5000 training images for each category. The image is flattened into a vector, so the total number of features is 32 x 32 x 3 = 3072.

The steps of PCA can be summarized as :
1. Take the covariance matrix (of features) and perform eigen value decomposition to find the eigen vectors and eigen values. SVD can also be done in place of EVD
2. Sort the eigen values in decreasing order and select the first $\mathrm{k}$ principal components. The value of $\mathrm{k}$ can be determined based on the percentage variance and identifying those that have high variance.
3. Now, take the entire image data containing feature vectors $\textbf{x}$ and subtract from it the mean image $\textit{x}$ 
4. Find the $\mathrm{k}$ coefficients for a given feature vector $\textbf{x}$ as $\phi_{k} = (\textbf{x}-\textit{x}).v_{k}$ where $v_{k}$ is the $\mathrm{k}^{th}$ principal vector.
5. Now reconstruct the image as: $\textbf{x} = \sum_{j=1}^{k}\phi_{k}v_{k}$.


The above steps have been clearly implemented in the code and we now discuss the results.

Visualization of the distribution of the first 10 features within the 10 different categories/labels. 
![Alt text](imgs/histo.PNG?raw=true "histo")

Sample images from each category :
![Alt text](imgs/original_images.PNG?raw=true "original_images")

Now we see the plot of percentage variance of the different eigenvectors:
![Alt text](imgs/variance_ex.png?raw=true "variance_ex")

As we can see, for the CIFAR-10 dataset, the first 10 eigenvectors contribute to ~60% of the variance in data. This means that the new 10-dimensional subspace represents 60% of the original data. However, during reconstruction I found that nearly 200 principal components produced effective reproduction of the original images with comparatively lesser loss. The reconstructed images are :

![Alt text](imgs/200_pc_recon_32_1.PNG?raw=true "200_pc_recon_32_1")
![Alt text](imgs/200_pc_recon_32_2.PNG?raw=true "200_pc_recon_32_2")

100 principal components used :

![Alt text](imgs/pca_recon_100.PNG?raw=true "pca_recon_100")

200 principal components used:

![Alt text](imgs/200_pc_recon_Fin.PNG?raw=true "200_pc_recon_Fin")

We can clearly see that the reconstructed image is of reasonable quality. Also, though there is loss of information in the images, the loss improves as more principal components are added. 

**LDA :**
The primary goal of LDA is to provide maximum separation between the data of different classes. So, while PCA works in an unsupervised setting and is unmindful of the labels of the data, LDA works with these labels and tries to find that subspace that can best separate the data of different categories.  
The procedure can be outlined as :
1. Find the mean vectors for each category of the data
2. Find the between class and within class scatter matrices
3. Find the eigenvectors and eigenvalues of the $\left(\mathrm{Scatter}_{within}\right)^{-1}\times \mathrm{Scatter}_{between}$
4. Follow steps 2 to 4 of PCA 

The major variation here is that the eigenvectors and eigen values computed with the scatter matrices show that the first 10 eigenvectors contribute to ~100% of the variance. This is a large difference compared to the analysis from PCA. I tried reconstructing the image and visualizing it, but it was not successful. 

![Alt text](imgs/lda_output_2.PNG?raw=true "lda_output_2")

One comparison that can be done between the PCA and LDA is, plotting the data to see how well the first 2 components separate the data. That is, we reduce the data to 2-dimensions by transforming the original data to the new eigenspace and see how well they are separated. We do the same for LDA.
**PCA Output**
![Alt text](imgs/PCA_Plot.png?raw=true "PCA_Plot")
**LDA Output**
![Alt text](imgs/LDA_Plot.png?raw=true "LDA_Plot")

We can observe that both PCA and LDA provide similar separation between the classes and it is hard to observe which one is best. However, in LDA the number of linear discriminants is utmost (number of classes - 1) and we can observe in our result also that only the first 9 eigen vectors sum up to produce cumulative variance of 100%

I have only done a simple analysis and there is more scope to explore the performance of these two and also other such algorithms

#TODO: 
1. Plot of eigen images
2. Reduction in loss as the number of principal components is increased